# Initializing a model

Example showing how to initialize a model with another model

In [19]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed

import os

import pandas as pd

import pybamm

os.chdir(pybamm.__path__[0] + "/..")


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Solve a model with a drive cycle

Load model

In [20]:
model = pybamm.lithium_ion.DFN()

Set up drive cycle

In [21]:
# import drive cycle from file
data_loader = pybamm.DataLoader()
drive_cycle = pd.read_csv(
    f"{data_loader.get_data('US06.csv')}", comment="#", header=None
).to_numpy()
# create interpolant
param = model.default_parameter_values
current_interpolant = pybamm.Interpolant(drive_cycle[:, 0], drive_cycle[:, 1], pybamm.t)
# set drive cycle
param["Current function [A]"] = current_interpolant

Create and run simulation with the updated parameters

In [22]:
sim_US06_1 = pybamm.Simulation(model, parameter_values=param)
sol_US06_1 = sim_US06_1.solve()

## Update initial conditions based on a solution and solve again

Starting with an initial SOC of 0%, pre-charge with CCCV, update the initial conditions, and solve again with the US06 drive cycle

In [23]:
experiment = pybamm.Experiment(
    ["Charge at 1 A until 3.9 V", "Hold at 3.9 V until 50 mA"],
)
sim_cccv = pybamm.Simulation(model, experiment=experiment)
sol_cccv = sim_cccv.solve(initial_soc=0)

# MODEL RE-INITIALIZATION: #############################################################
# Now initialize the model with the solution of the charge, and then discharge with
# the US06 drive cycle
# We could also do this inplace by setting inplace to True, which modifies the original
# model in place
new_model = model.set_initial_conditions_from(sol_cccv, inplace=False)
########################################################################################

sim_US06_2 = pybamm.Simulation(new_model, parameter_values=param)
sol_US06_2 = sim_US06_2.solve()

Plot both solutions, we can clearly see the difference now that initial conditions have been updated

In [24]:
pybamm.dynamic_plot(
    [sol_US06_1, sol_US06_2], labels=["Default initial conditions", "Fully charged"]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

## Initialize using a different model

We can also initialize the model using the solution of a different model

In [25]:
spm = pybamm.lithium_ion.SPM()
sim_spm_cccv = pybamm.Simulation(spm, experiment=experiment)
sol_spm_cccv = sim_spm_cccv.solve(initial_soc=0)

# MODEL RE-INITIALIZATION: #############################################################
# Now initialize the model with the solution of the charge, and then discharge with
# the US06 drive cycle
# We could also do this inplace by setting inplace to True, which modifies the original
# model in place
new_dfn = model.set_initial_conditions_from(sol_spm_cccv, inplace=False)
########################################################################################

sim_US06_3 = pybamm.Simulation(new_dfn, parameter_values=param)
sol_US06_3 = sim_US06_3.solve()

Now the model initialized by the DFN and the model initialized by the SPM give the same solution

In [26]:
pybamm.dynamic_plot(
    [sol_US06_1, sol_US06_2, sol_US06_3],
    labels=[
        "Default initial conditions",
        "Fully charged (from DFN)",
        "Fully charged (from SPM)",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

## References

The relevant papers for this notebook are:

In [27]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Von DAG Bruggeman. Berechnung verschiedener physikalischer konstanten von heterogenen substanzen. i. dielektrizitätskonstanten und leitfähigkeiten der mischkörper aus isotropen substanzen. Annalen der physik, 416(7):636–664, 1935.
[3] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[4] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[5] 